In [71]:
import numpy as np
import scipy as sps
import pandas as pd
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [77]:
data = pd.read_csv("dataset/csv_dataset_with_locations.txt", sep="|")

/Users/massimo/anaconda/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
data.head()

,id,full_text,formatted_text,event_id,created_at,user_id,in_reply_to_status_id,hash_tags,user_mentions,retweet_count,favorite_count,possibly_sensitive,place_name,place_type,country_code,coordinates,is_fake
0,615868085090676737,"Walmart bakes ISIS cake, rejects Confederate f...","Walmart bakes ISIS cake, rejects Confederate f...",E695,Tue Jun 30 13:02:48 +0000 2015,1.643046e+09,NaN,NaN,NaN,0,0,False,NaN,NaN,NaN,NaN,0.0
1,14619605813,UK bans doctor who linked autism to vaccine: B...,UK bans doctor who linked autism to vaccine: B...,TM859,Mon May 24 12:23:36 +0000 2010,1.247795e+08,NaN,NaN,NaN,0,0,False,NaN,NaN,NaN,NaN,0.0
2,644875384622829568,Doritos unveils rainbow chips to support LGBT ...,Doritos unveils rainbow chips to support LGBT ...,E391,Fri Sep 18 14:07:27 +0000 2015,2.297099e+07,NaN,NaN,NaN,3,6,False,NaN,NaN,NaN,NaN,0.0
3,647483888684498944,Mike Huckabee Is Having A Meltdown About Those...,Mike Huckabee Is Having A Meltdown About Those...,E391,Fri Sep 25 18:52:43 +0000 2015,1.179711e+09,NaN,NaN,NaN,1,0,False,NaN,NaN,NaN,NaN,0.0
4,608681780812652544,The only good news about the McKinney pool par...,The only good news about the McKinney pool par...,E768,Wed Jun 10 17:06:59 +0000 2015,2.234826e+09,NaN,NaN,2347049341,0,0,False,NaN,NaN,NaN,NaN,0.0


In [53]:
data.iloc[1880]

id                                                             21999278523
full_text                http://itanil.com/2010/08/24/tiger-woods-and-w...
formatted_text                                                         NaN
event_id                                                            TM2525
created_at                                  Tue Aug 24 13:09:34 +0000 2010
user_id                                                        6.17129e+07
in_reply_to_status_id                                                  NaN
hash_tags                                                              NaN
user_mentions                                                          NaN
retweet_count                                                            0
favorite_count                                                           0
possibly_sensitive                                                   False
place_name                                                             NaN
place_type               

In [78]:
txt = data["formatted_text"].values

txt[pd.isnull(txt)] = "" # remove NaN

from string import punctuation
def clean_text(arr_text):
    for i in range(len(arr_text)):
        tweet = arr_text[i]
        tweet = tweet.lower()
        tweet = tweet.translate(str.maketrans('', '', punctuation))
        arr_text[i] = tweet
        
clean_text(txt)

In [79]:
labels = list(range(len(txt)))

In [80]:
source = []
for i in range(len(txt)):
    source+=[TaggedDocument(words=txt[i].split(), tags=[labels[i]])]

In [81]:
model = Doc2Vec(min_count=0, window=10, alpha=0.025, vector_size=30, sample=1e-6, negative=3, workers=8) # vector_size: max 100
model.build_vocab(source)
#model = Doc2Vec(documents, size=100, window=8, min_count=5, workers=4)
#model = gensim.models.Doc2Vec(documents, dm = 0, , size= 20, min_alpha=0.025, min_count=0)


In [83]:
model.train(source, total_examples=len(source), epochs=20)

In [86]:
model.most_similar('doctor')

/Users/massimo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('autism', 0.9785845875740051),
 ('linking', 0.9741197228431702),
 ('linked', 0.9725444316864014),
 ('vaccine', 0.9695267081260681),
 ('britains', 0.9593044519424438),
 ('quakercarecom', 0.946308970451355),
 ('britain', 0.9282881021499634),
 ('persuaded', 0.9022834897041321),
 ('bans', 0.8638392090797424),
 ('meas', 0.8567782044410706)]

In [90]:
model.most_similar('islam')

/Users/massimo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('middle', 0.966205894947052),
 ('tennessee', 0.961780309677124),
 ('ntb', 0.9563257694244385),
 ('god”', 0.9502483606338501),
 ('nonmuslim', 0.9439868330955505),
 ('forcing', 0.9377407431602478),
 ('districts', 0.9294548034667969),
 ('praise', 0.9278393983840942),
 ('lessons', 0.9224681258201599),
 ('comply', 0.9197720289230347)]

In [91]:
model.save('./model_doc2vec.d2v')

In [93]:
#model = Doc2Vec.load('./model_doc2vec.d2v')